<a href="https://colab.research.google.com/github/pkraison/Initializ/blob/master/CompressedSensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# make sure you've got the following packages installed
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as spopt
import scipy.fftpack as spfft
import scipy.ndimage as spimg
import cvxpy as cvx

In [ ]:
plt.style.use('classic')

In [ ]:
# generate some data with noise
x = np.sort(np.random.uniform(0, 10, 15))
y = 3 + 0.2 * x + 0.1 * np.random.randn(len(x))

In [ ]:
# find L1 line fit
l1_fit = lambda x0, x, y: np.sum(np.abs(x0[0] * x + x0[1] - y))
xopt1 = spopt.fmin(func=l1_fit, x0=[1, 1], args=(x, y))

# find L2 line fit
l2_fit = lambda x0, x, y: np.sum(np.power(x0[0] * x + x0[1] - y, 2))
xopt2 = spopt.fmin(func=l2_fit, x0=[1, 1], args=(x, y))

Optimization terminated successfully.
         Current function value: 0.738900
         Iterations: 54
         Function evaluations: 102
Optimization terminated successfully.
         Current function value: 0.066883
         Iterations: 49
         Function evaluations: 94


In [ ]:
# adjust data by adding outlyers
y2 = y.copy()
y2[3] += 4
y2[13] -= 3

# refit the lines
xopt12 = spopt.fmin(func=l1_fit, x0=[1, 1], args=(x, y2))
xopt22 = spopt.fmin(func=l2_fit, x0=[1, 1], args=(x, y2))

Optimization terminated successfully.
         Current function value: 7.669809
         Iterations: 58
         Function evaluations: 111
Optimization terminated successfully.
         Current function value: 22.058906
         Iterations: 54
         Function evaluations: 102


In [ ]:
# sum of two sinusoids
n = 5000
t = np.linspace(0, 1/8, n)
y = np.sin(1394 * np.pi * t) + np.sin(3266 * np.pi * t)
yt = spfft.dct(y, norm='ortho')

#plt.plot(y)
#plt.plot(yt)

In [ ]:
# extract small sample of signal
m = 500 # 10% sample
ri = np.random.choice(n, m, replace=False) # random sample of indices
ri.sort() # sorting not strictly necessary, but convenient for plotting
t2 = t[ri]
y2 = y[ri]

In [ ]:
# create idct matrix operator
A = spfft.idct(np.identity(n), norm='ortho', axis=0)
A = A[ri]

# do L1 optimization
vx = cvx.Variable(n)
objective = cvx.Minimize(cvx.norm(vx, 1))
constraints = [A*vx == y2]
prob = cvx.Problem(objective, constraints)
result = prob.solve(verbose=True)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Nov 09 01:03:12 PM: Your problem has 5000 variables, 1 constraints, and 0 parameters.
(CVXPY) Nov 09 01:03:13 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 09 01:03:13 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 09 01:03:13 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 09 01:03:13 PM: Compiling problem (target solver=ECOS).
(CVXPY) Nov 09 01:03:13 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

/usr/local/lib/python3.7/dist-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


(CVXPY) Nov 09 01:03:14 PM: Applying reduction ECOS
(CVXPY) Nov 09 01:03:15 PM: Finished problem compilation (took 2.376e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Nov 09 01:03:15 PM: Invoking solver ECOS  to obtain a solution.
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Nov 09 01:03:57 PM: Problem status: optimal
(CVXPY) Nov 09 01:03:57 PM: Optimal value: 3.485e+02
(CVXPY) Nov 09 01:03:57 PM: Compilation took 2.376e+00 seconds
(CVXPY) Nov 09 01:03:57 PM: Solver (including time spent in interface) took 4.175e+01 seconds
